In [1]:
import os
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
from random import randint
from numpy.random import rand
import flask
import dash
from dash.dependencies import Input, Output, State
from random import randint
import landing_page
import person_page
import household_page
import dash_table

/Users/ericcriteser/anaconda2/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a release (2.7.7+) that supports hmac.compare_digest as soon as possible.
  utils.PersistentlyDeprecated2018,


In [2]:
#pages
page = landing_page.get_layout()
person = person_page.get_person_layout()
household = household_page.get_household_layout()

In [3]:
def print_items(items):
    check_dict = {'ceiling': 0, 'bathroom': 0, 'refrigerator': 0, 'computer': 0, 'tv': 0, 'household_id': 0}
    cols = ['ceiling','bathroom','refrigerator','computer','tv']
    items = items
    for i in items:
        #print('here', type(i))
        if i not in cols:
            #print('now_here')
            check_dict['household_id'] = items[-1:][0]
        elif i in check_dict.keys():
            check_dict[i] = 1
        else:
            pass
    return check_dict

In [8]:
app = dash.Dash(__name__)

app.config['suppress_callback_exceptions']=True

hh_id = 111111

z_df =[]

persons_list = []
household_list = []
check_list = []

hh_cols = ['roof_material', 'wall_material', 'floor_material', 'overall_condition',
           'house_ownership', 'region', 'urban', 'thrash_disposal', 'rooms',
           'num_tablets', 'num_bedrooms', 'num_mobile_phones','elec_provider',
           'water_supply', 'cooking_supply', 'sewage', 'household_id']

checklist_cols = ['ceiling', 'bathroom', 'refrigerator', 'computer', 'tv']
check_dict = {'ceiling': 0, 'bathroom': 0, 'refrigerator': 0, 'computer': 0, 'tv': 0, 'household_id': 0}

t_df = pd.read_csv('temp_df.csv')

app.layout = html.Div(
                html.Center([
                    dcc.Location(id='url', refresh=False),
                    dcc.Link('Select Household Id', href='/', style = {'margin':'30px', 'font-size': '12px'}),
                    dcc.Link('Add Person', href='/person', style = {'margin':'30px', 'font-size': '12px'}),
                    dcc.Link('Edit Household', href = '/household', style = {'margin':'30px', 'font-size': '12px'}),
                    dcc.Link('View Household', href = '/view_household', style = {'margin':'30px', 'font-size': '12px'}),
                    html.Center(html.H3(id = 'hh_id', children = [''], style = {'margin-top':'30px', 'margin-bottom':'30px'})),
                    # render page here
                    html.Div(id='page-content'),
                    html.Div(id = 't_holder')
                ])
            )

#set page content from navbar
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/person':
        return html.Div([
            person,
        ]) 
    elif pathname == '/household':
        return html.Div([
            household,
        ]) 
    elif pathname == '/view_household':
        return html.Div([
            dash_table.DataTable(
                id='table',
                columns=[{"name": i, "id": i} for i in t_df.columns],
                data=t_df.to_dict('records'),
            ),
            html.Br(),
            html.Br(),
            html.Center(html.Div(html.Button('Get Prediction', id='predict_submit', n_clicks=0))),
            html.Br(),
            html.Br(),
            html.Center(html.H3(id = 'prediction', children = ['']))
            
        ])
    #else return main page
    else:
        return html.Div([
            page,
        ])
    
#Landing Page    
#go to person on set hh_id valid id
@app.callback(
    dash.dependencies.Output('url', 'pathname'),
    [dash.dependencies.Input('hh_id', 'children')
    ])
def load_new_info(children):
    if children != 'Household Id: Select Id':
        return '/person' 
    else:
        raise dash.exceptions.PreventUpdate()
    
#Landing Page
#set hh_id on 'New Household ID' os assign id   
@app.callback(
    dash.dependencies.Output('hh_id', 'children'),
     [dash.dependencies.Input('id_dropdown', 'value'),
      dash.dependencies.Input('submit-val', 'n_clicks')
    ])
def load_new_info(value, n_clicks):
    if n_clicks > 0:
        hh_id = np.random.randint(100000, 200000)
    else:
        hh_id = value
    return 'Household Id: {}' .format(hh_id)

#Person Page
@app.callback(
    dash.dependencies.Output('person_rec', 'children'),
    [dash.dependencies.Input('add_person', 'n_clicks')],
    state=[State('gender_dropdown', 'value'), State('age_dropdown', 'value'), State('school_dropdown', 'value'),
           State('disabled_dropdown', 'value'), State('person_rec', 'value'), State('hh_id', 'children')])
def load_new_info(n_clicks, gender_dropdown, age_dropdown, school_dropdown, disabled_dropdown, person_rec, children):
    hh_id = np.array(str(children).split(' ')[-1:][0])
    person_items = [gender_dropdown, age_dropdown, school_dropdown, disabled_dropdown]
    if None in person_items:
        return 'All Fields Required'
    else:
        ###save person data
        person_items.append(str(hh_id))
        persons_list.append(person_items)
        return '{} year old {} added to {} person list.' .format(age_dropdown, gender_dropdown, len(persons_list))
    
#Household Page    
@app.callback(
    dash.dependencies.Output('t_holder', 'children'),###
    [dash.dependencies.Input('save_household', 'n_clicks')],
    state = [State('roof_material_dropdown', 'value'), State('wall_material_dropdown', 'value'), 
             State('floor_material_dropdown', 'value'), State('overall_dropdown', 'value'), State('ownership_dropdown', 'value'),
             State('region_dropdown', 'value'), State('rural_dropdown', 'value'), State('thrash_dropdown', 'value'),
             State('rooms_dropdown', 'value'), State('num_tablets_dropdown', 'value'), State('num_bedrooms_dropdown', 'value'),
             State('num_mobile_dropdown', 'value'), State('elec_provider_dropdown', 'value'), State('water_supply_dropdown', 'value'),
             State('cooking_supply_dropdown', 'value'), State('sewage_dropdown', 'value'), State('checklist_items', 'values'),
                                                              State('hh_id', 'children')]
)
#def load_new_info(n_clicks, roof_material_dropdown, wall_material_dropdown, floor_material_dropdown, overall_dropdown,
                  #ownership_dropdown, region_dropdown, rural_dropdown, thrash_dropdown, rooms_dropdown,
                  #num_tablets_dropdown, num_bedrooms_dropdown, num_mobile_dropdown, elec_provider_dropdown,
                  #water_supply_dropdown, cooking_supply_dropdown, sewage_dropdown, checklist_items, hh_id):
    #household_items = [roof_material_dropdown, wall_material_dropdown, floor_material_dropdown, overall_dropdown,
                       #ownership_dropdown, region_dropdown, rural_dropdown, thrash_dropdown, rooms_dropdown,
                       #num_tablets_dropdown, num_bedrooms_dropdown, num_mobile_dropdown, elec_provider_dropdown,
                       #water_supply_dropdown, cooking_supply_dropdown, sewage_dropdown]
    
def load_new_info(n_clicks, roof_material, wall_material, floor_material, overall,
                  ownership, region, rural, thrash, rooms,
                  num_tablets, num_bedrooms, num_mobile, elec_provider,
                  water_supply, cooking_supply, sewage, checklist_items, hh_id):
    household_items = [roof_material, wall_material, floor_material, overall,
                       ownership, region, rural, thrash, rooms,
                       num_tablets, num_bedrooms, num_mobile, elec_provider,
                       water_supply, cooking_supply, sewage]
    hh_id = np.array(str(hh_id).split(' ')[-1:][0])
    checklist_items.append(str(hh_id))
    t_dict = print_items(checklist_items)
    check_list.append(t_dict)
    if None in household_items:
        print('None Present')
        raise dash.exceptions.PreventUpdate()
    else:
        household_items.append(str(hh_id))
        household_list.append(household_items)
        return ' ' 

#view household/get predictions page
@app.callback(
    dash.dependencies.Output('prediction', 'children'),
    [dash.dependencies.Input('predict_submit', 'n_clicks')
    ])
def load_new_info(n_clicks):
    if n_clicks > 0:
        p_df = pd.DataFrame(persons_list, columns = ['gender', 'age', 'years_school', 'disable_person', 'household_id'])
        h_df = pd.DataFrame(household_list, columns = hh_cols)
        c_df = pd.DataFrame([check_list[0].values()], columns = check_dict.keys())
        ph_df = h_df.merge(p_df, on = 'household_id')
        hh_df = ph_df.merge(c_df, on = 'household_id')
        z_df.append(hh_df)
        #print(hh_df)
        return 'Category 3'
    else:
        raise dash.exceptions.PreventUpdate()
        

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8124)

Running on http://127.0.0.1:8124/
Running on http://127.0.0.1:8124/
Running on http://127.0.0.1:8124/
Running on http://127.0.0.1:8124/
Debugger PIN: 473-442-289
Debugger PIN: 473-442-289
Debugger PIN: 473-442-289
Debugger PIN: 473-442-289


In [9]:
z_df

df = z_df[0].copy()

df.gender = [1 if g == 'female' else 0 for g in df.gender]
df.rename(columns={'gender': 'female'}, inplace = True)
df.urban = [1 if u == 'urban' else 0 for u in df.urban]
df['num_persons'] = len(df)
df['males_under_12'] = len(df.query('female == 0 and age < 12'))
df['males_over_12'] = len(df.query('female == 0 and age > 12'))
df['num_males'] = len(df.query('female == 0'))
df['females_under_12'] = len(df.query('female == 1 and age < 12'))
df['females_over_12'] = len(df.query('female == 1 and age > 12'))
df['num_females'] = len(df.query('female == 1'))
df['persons_under_12'] = len(df.query('age < 12'))
df['persons_over_12'] = len(df.query('age > 12'))
df['under_10'] = len(df.query('age < 10'))
df['num_minors'] = len(df.query('age < 18'))
df['num_adults'] = len(df.query('age > 18'))
df['num_over_65'] = len(df.query('age > 65'))
df['mean_edu_adults'] = df.years_school.sum()/ df.num_persons
df['new_calc_dep'] = (df.num_over_65 + df.persons_under_12)/ df.num_persons

df['overcrowding_persons_per_room'] = [x/y for x,y in zip(df['num_persons'],df['num_bedrooms'])]
df['overcrowding_by_bedrooms'] = [1 if x/y > 2 else 0 for x,y in zip(df['num_persons'],df['num_bedrooms'])]
df['overcrowding_by_rooms'] = [1 if x/y > 2 else 0 for x,y in zip(df['num_persons'],df['rooms'])]

#drop downs to binary
df['roof_material_{}' .format(str(df.roof_material.unique()[0]))] = 1
df['floor_material_{}' .format(str(df.floor_material.unique()[0]))] = 1
df['wall_material_{}' .format(str(df.wall_material.unique()[0]))] = 1
df['house_ownership_{}' .format(str(df.house_ownership.unique()[0]))] = 1
df['region_{}' .format(str(df.region.unique()[0]))] = 1
df['thrash_disposal_{}' .format(str(df.thrash_disposal.unique()[0]))] = 1
df['elec_provider_{}' .format(str(df.elec_provider.unique()[0]))] = 1
df['water_supply_{}' .format(str(df.water_supply.unique()[0]))] = 1
df['cooking_supply_{}' .format(str(df.cooking_supply.unique()[0]))] = 1
df['sewage_{}' .format(str(df.sewage.unique()[0]))] = 1

to_drop = ['roof_material', 'wall_material', 'floor_material', 'house_ownership', 'region',  'thrash_disposal',
           'elec_provider', 'sewage', 'cooking_supply', 'water_supply']

df.drop(to_drop, axis = 1, inplace = True)

old_df_cols = ['overcrowding_by_bedrooms', 'rooms', 'overcrowding_by_rooms',
       'bathroom', 'refrigerator', 'num_tablets', 'males_under_12',
       'males_over_12', 'num_males', 'females_under_12', 'females_over_12',
       'num_females', 'persons_under_12', 'persons_over_12', 'num_persons',
       'years_school', 'ceiling', 'disable_person', 'female', 'under_10',
       'num_minors', 'num_adults', 'num_over_65', 'mean_edu_adults',
       'num_bedrooms', 'overcrowding_persons_per_room', 'computer', 'tv',
       'num_mobile_phones', 'urban', 'age', 'Target', 'new_calc_dep',
       'overall_condition', 'roof_material_fiber_cement',
       'roof_material_metal', 'roof_material_natural_fibers_roof',
       'roof_material_other_roof', 'elec_provider_cooperative',
       'elec_provider_none_electricity', 'elec_provider_private_plant',
       'elec_provider_public', 'wall_material_block_brick',
       'wall_material_natural_fibers', 'wall_material_other_wall',
       'wall_material_prefabricated_cement',
       'wall_material_waste_material', 'wall_material_wood',
       'wall_material_wood_zinc_absbesto', 'wall_material_zink',
       'house_ownership_other_house', 'house_ownership_own',
       'house_ownership_own_installments', 'house_ownership_precarious',
       'house_ownership_rented', 'region_brunca', 'region_central',
       'region_chorotega', 'region_huetar_atlantic', 'region_huetar_norte',
       'region_pacific_central', 'water_supply_inside',
       'water_supply_outside', 'water_supply_water_none',
       'cooking_supply_charcoal', 'cooking_supply_electricity',
       'cooking_supply_energy_none', 'cooking_supply_gas',
       'floor_material_cement', 'floor_material_natural_material_floor',
       'floor_material_none_floor', 'floor_material_other_floor',
       'floor_material_tile', 'floor_material_wood_floor',
       'sewage_letrine', 'sewage_no_toilet', 'sewage_other_sewage',
       'sewage_septic_tank', 'sewage_sewer_cesspool',
       'thrash_disposal_burning', 'thrash_disposal_hollow_buried',
       'thrash_disposal_other_disposal', 'thrash_disposal_tanker truck',
       'thrash_disposal_throwing_space']

cols_not_in = [c for c in old_df_cols if c not in df.columns.values]
#cols_not_in

for c in cols_not_in:
    df[c] = 0
    
    #has house_id and Target
len(df.columns.values)

df

,overall_condition,urban,rooms,num_tablets,num_bedrooms,num_mobile_phones,household_id,female,age,years_school,...,floor_material_tile,floor_material_wood_floor,sewage_letrine,sewage_no_toilet,sewage_septic_tank,sewage_sewer_cesspool,thrash_disposal_burning,thrash_disposal_hollow_buried,thrash_disposal_tanker truck,thrash_disposal_throwing_space
0,4,0,3,0,1,2,136371,1,16,9,...,0,0,0,0,0,0,0,0,0,0
1,4,0,3,0,1,2,136371,0,21,8,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 85 columns):
overall_condition                        2 non-null int64
urban                                    2 non-null int64
rooms                                    2 non-null int64
num_tablets                              2 non-null int64
num_bedrooms                             2 non-null int64
num_mobile_phones                        2 non-null int64
household_id                             2 non-null object
female                                   2 non-null int64
age                                      2 non-null int64
years_school                             2 non-null int64
disable_person                           2 non-null int64
ceiling                                  2 non-null int64
tv                                       2 non-null int64
bathroom                                 2 non-null int64
computer                                 2 non-null int64
refrigerator                